### Import Libraries

In [1]:
# Import the required packages and libraries
import datetime
import os
from dotenv import load_dotenv 
from pathlib import Path
import nest_asyncio

nest_asyncio.apply()

### Import Scripts

In [2]:
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

### Import Envrionment Variables

In [3]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [4]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [5]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

### Download data from modelindex api

In [6]:
# Connecting to ModelIndex APIs 
model_data = ModelIndex(url=model_index_url, auth_client=auth_client, session=auth_client.session)

In [7]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

[{'Idx': 0, 'Uri': 'http://opcfoundation.org/UA/'},
 {'Idx': 1, 'Uri': 'http://prediktor.no/apis/ua/'},
 {'Idx': 2, 'Uri': 'urn:prediktor:UIDEV-W2022-04:Scatec'},
 {'Idx': 3, 'Uri': 'http://scatecsolar.com/EG-AS'},
 {'Idx': 4, 'Uri': 'http://scatecsolar.com/Enterprise'},
 {'Idx': 5, 'Uri': 'http://scatecsolar.com/JO-GL'},
 {'Idx': 6, 'Uri': 'http://prediktor.no/PVTypes/'},
 {'Idx': 7, 'Uri': 'http://powerview.com/enterprise'}]

In [8]:
# Types of Objects
object_types_json = model_data.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

,Id,Name,BrowseName,Props,Vars
0,6:0:1052,ICalcTracker,ICalcTracker,[],[]
1,6:0:1061,EquipmentEventType,EquipmentEventType,[],[]
2,6:0:1128,EnergyAndPowerMeterEventType,EnergyAndPowerMeterEventType,[],[]
3,6:0:1263,EnergyAndPowerMeterCommLossEventType,EnergyAndPowerMeterCommLossEventType,[],[]
4,6:0:1266,EnergyAndPowerMeterErrorEventType,EnergyAndPowerMeterErrorEventType,[],[]
...,...,...,...,...,...
239,6:0:1050,OpcUaHiveModule,OpcUaHiveModule,[],[]
240,6:0:1057,BaseHiveModuleNoItemsType,BaseHiveModuleNoItemsType,[],[]
241,6:0:1067,LoggerHiveModule,LoggerHiveModule,[],[]
242,6:0:1055,EquipmentTemplateDefinitionType,EquipmentTemplateDefinitionType,[],[]


In [9]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

,Id,Name
0,6:0:1052,ICalcTracker
1,6:0:1061,EquipmentEventType
2,6:0:1128,EnergyAndPowerMeterEventType
3,6:0:1263,EnergyAndPowerMeterCommLossEventType
4,6:0:1266,EnergyAndPowerMeterErrorEventType
...,...,...
118,6:0:1050,OpcUaHiveModule
119,6:0:1057,BaseHiveModuleNoItemsType
120,6:0:1067,LoggerHiveModule
121,6:0:1055,EquipmentTemplateDefinitionType


In [10]:
# To get typeId by type name of an object
object_type_id = model_data.get_object_type_id_from_name("SiteType")
object_type_id

'6:0:1009'

In [11]:
# To get the objects of a type
sites_json = model_data.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

['EG-AS', 'JO-GL']

In [12]:
# Analytics helper
sites.variables_as_dataframe()

,Id,Type,Name,VariableId,VariableName,VariableIdSplit
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Alarms.CommLossPlantDevice,CommLossPlantDevice,{'Id': 'Enterprise.EG-AS.Alarms.CommLossPlantD...
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Signals.PPC.IsCurtailment,PPC.IsCurtailment,{'Id': 'Enterprise.EG-AS.Signals.PPC.IsCurtail...
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Signals.State.IsDay,State.IsDay,"{'Id': 'Enterprise.EG-AS.Signals.State.IsDay',..."
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Parameters.ContractDuration,ContractDuration,{'Id': 'Enterprise.EG-AS.Parameters.ContractDu...
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Parameters.RegionKey,RegionKey,{'Id': 'Enterprise.EG-AS.Parameters.RegionKey'...
...,...,...,...,...,...,...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.Weather.Irradiati...,Weather.IrradiationDiffuseHorizontal,{'Id': 'Enterprise.JO-GL.Signals.Weather.Irrad...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.Weather.Irradiati...,Weather.IrradiationHorizontal,{'Id': 'Enterprise.JO-GL.Signals.Weather.Irrad...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.Weather.Irradiati...,Weather.IrradiationInCline,{'Id': 'Enterprise.JO-GL.Signals.Weather.Irrad...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.ACActiveEnergy.Total,ACActiveEnergy.Total,{'Id': 'Enterprise.JO-GL.Signals.ACActiveEnerg...


In [13]:
sites.list_of_ids()

['3:1:Enterprise.EG-AS', '5:1:Enterprise.JO-GL']

In [14]:
# Selecting the single site
site_id = sites.list_of_ids()[0]
site_id

'3:1:Enterprise.EG-AS'

In [15]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model_data.get_object_descendants(
    "StringSetType", [site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

,Id,Name,Type,Props,Vars
0,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01,EG-AS-TS01-I01-SM01-CH01,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '01'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
1,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH02,EG-AS-TS01-I01-SM01-CH02,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '02'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH03,EG-AS-TS01-I01-SM01-CH03,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '03'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
3,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH04,EG-AS-TS01-I01-SM01-CH04,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '04'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
4,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH05,EG-AS-TS01-I01-SM01-CH05,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '05'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
...,...,...,...,...,...
2933,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH05,EG-AS-TS11-I22-SM13-CH05,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '05'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2934,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH06,EG-AS-TS11-I22-SM13-CH06,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '06'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2935,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH07,EG-AS-TS11-I22-SM13-CH07,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '07'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2936,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH08,EG-AS-TS11-I22-SM13-CH08,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '08'}, ...","[{'DisplayName': 'StringDisconnected', 'Id': '..."


In [16]:
# All inverter data for the site
inverter_json = model_data.get_object_descendants(
    "InverterType", [site_id], "PV_Assets"
)
inverters = AnalyticsHelper(inverter_json)

In [17]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model_data.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

,Id,Name,Type,VariableId,VariableName,VariableIdSplit
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,TrackerOutOfPos,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,State.HasHighSeverityAlarm,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,State.HasMediumSeverityAlarm,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,State.HasLowSeverityAlarm,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,CommLoss,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
...,...,...,...,...,...,...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,TrackingLimitWestAngle,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,MotorPressure,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,Category,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,StateCode,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...


### Download data from the opc ua api

In [18]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initating the OPC UA API with a fixed namespace list
opc_data = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list, auth_client=auth_client)

In [19]:
# Live value data of trackers
live_value = opc_data.get_values(
    trackers.variables_as_list(["AngleMeasured"])
)
live_value

[{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Signals.AngleMeasured',
  'Namespace': 3,
  'IdType': 1,
  'Timestamp': '2024-09-26T11:49:01.7273647Z',
  'Value': 0,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR002.Signals.AngleMeasured',
  'Namespace': 3,
  'IdType': 1,
  'Timestamp': '2024-09-26T11:49:01.7334401Z',
  'Value': 20.00982,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Signals.AngleMeasured',
  'Namespace': 3,
  'IdType': 1,
  'Timestamp': '2024-09-26T11:49:01.7273647Z',
  'Value': 0,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Signals.AngleMeasured',
  'Namespace': 3,
  'IdType': 1,
  'Timestamp': '2024-09-26T11:49:01.7273647Z',
  'Value': 0,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'Enterprise.EG-AS.S1.MMST

In [20]:
# 1 day aggregated historical data
one_day_historical_data = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=string_sets_for_first_park.variables_as_list(["DCPower"])
)
one_day_historical_data

,Timestamp,ValueType,Value,StatusCode,StatusSymbol,IdType,Id,Namespace
0,2024-08-27T13:49:34.283241Z,Double,2420.122314,1,Good,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
1,2024-08-27T13:50:34.283241Z,Double,2926.278809,1,Good,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
2,2024-08-27T13:51:34.283241Z,Double,2981.627930,1,Good,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
3,2024-08-27T13:52:34.283241Z,Double,3375.974365,1,Good,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
4,2024-08-27T13:53:34.283241Z,Double,3476.256348,1,Good,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
...,...,...,...,...,...,...,...,...
4230715,2024-08-28T13:44:34.283241Z,Double,6677.901855,1,Good,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
4230716,2024-08-28T13:45:34.283241Z,Double,6677.901855,1,Good,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
4230717,2024-08-28T13:46:34.283241Z,Double,5503.101562,1,Good,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
4230718,2024-08-28T13:47:34.283241Z,Double,3167.954834,1,Good,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3


In [21]:
# 1 day raw historical data
one_day_raw_historical_data = opc_data.get_raw_historical_values(
    start_time = datetime.datetime(2024, 7, 13, 00, 00),
    end_time = datetime.datetime(2024, 7, 13, 23, 59),
    variable_list=string_sets_for_first_park.variables_as_list(["DCPower"])
)
one_day_raw_historical_data

,Timestamp,ValueType,Value,StatusCode.Code,StatusCode.Symbol,IdType,Id,Namespace
0,2024-07-13T00:00:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
1,2024-07-13T00:01:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
2,2024-07-13T00:02:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
3,2024-07-13T00:03:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
4,2024-07-13T00:04:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01.Sign...,3
...,...,...,...,...,...,...,...,...
4227777,2024-07-13T23:54:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
4227778,2024-07-13T23:55:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
4227779,2024-07-13T23:56:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
4227780,2024-07-13T23:57:00Z,Double,0.0,1.083507e+09,UncertainSubNormal,1,Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH09.Sign...,3
